<a href="https://colab.research.google.com/github/sanjayparajuli/colab/blob/main/sqlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install pysqlite3
from pysqlite3 import dbapi2 as sqlite3
print(sqlite3.sqlite_version)
import pandas as pd
from IPython.display import display, HTML

3.37.2


In [48]:
dbname = 'music_streaming_1.db'
def printSqlResults(cursor, tblName):
  try:
    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
    display(HTML("<b><font color=Green> " + tblName + "</font></b>" + df.to_html(index=False)))
  except:
    pass

def runSql(caption, query):
  conn = sqlite3.connect(dbname)
  cursor = conn.cursor()
  cursor.execute(query)
  printSqlResults(cursor, caption)
  conn.close()

def runStepByStepSql(query, fromline):
  lines = query.strip().split('\n')
  for lineindx in range(fromline, len(lines)):
    partial_query = '\n'.join(lines[:lineindx])
    caption = 'Query till line:' + partial_query
    runSql(caption, partial_query + ':')


In [49]:
conn = sqlite3.connect(dbname)
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS Users (
  user_id INTEGER PRIMARY KEY,
  name VARCHAR(100) NOT NULL,
  email VARCHAR(100) NOT NULL UNIQUE
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Songs (
  song_id INTEGER PRIMARY KEY,
  title VARCHAR(100) NOT NULL,
  artist VARCHAR(100) NOT NULL,
  genre VARCHAR(100)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Listens (
  listen_id INTEGER PRIMARY KEY,
  user_id INTEGER NOT NULL,
  song_id INTEGER NOT NULL,
  rating FLOAT,
  listen_time TIMESTAMP,
  FOREIGN KEY (user_id) REFERENCES User(user_id),
  FOREIGN KEY (song_id) REFERENCES Songs(song_id)

);
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS Recommendations (
  user_id INTEGER NOT NULL,
  song_id INTEGER NOT NULL,
  recommendation_id INTEGER PRIMARY KEY,
  recommendation_time TIMESTAMP,
  FOREIGN KEY (user_id) REFERENCES Users(user_id),
  FOREIGN KEY (song_id) REFERENCES Songs (song_id)
);
""")
conn.commit()
conn.close()

In [50]:
conn = sqlite3.connect(dbname)
sqlite3.enable_callback_tracebacks(True)
cursor = conn.cursor()
cursor.execute('delete from Songs;')
cursor.execute('delete from Users;')
cursor.execute('delete from Listens;')
cursor.execute('delete from Recommendations;')

cursor.execute("""
INSERT INTO Users (user_id, name, email)
VALUES
  (1, 'Mickey', 'mickey@example.com'),
  (2, 'Mine', 'minnie@example.com'),
  (3, 'Daffy', 'daffy@example.com'),
  (4, 'Pluto', 'pluto@example.com');
""")

cursor.execute("""
INSERT INTO Songs(song_id, title, artist, genre)
VALUES
(1, 'Evermore', 'Taylor Swift', 'Pop'),
(2, 'Willow', 'Taylor Swift', 'Pop'),
(3, 'Shape of You', 'Ed Sheeran', 'Rock'),
(4, 'Photograph', 'Ed Sheeran', 'Rock'),
(5, 'Shivers', 'Ed Sheeran', 'Rock'),
(6, 'Yesterday', 'Beatles', 'Classic'),
(7, 'Yellow Submarine', 'Beatles', 'Classic'),
(8, 'Hey Jude', 'Beatles', 'Classic'),
(9, 'Bad Blood', 'Taylor Swift', 'Rock'),
(10, 'DJ Mix', 'DJ', NULL);

""")
cursor.execute("""
INSERT INTO Listens (listen_id, user_id, song_id, rating, listen_time)
VALUES
(1, 1, 1, 4.5, '2024-08-30 14:35:00'),
(2, 1, 2, 4.2, NULL),
(3, 1, 6, 3.9, '2024-08-29 10:15:00'),
(4, 2, 2, 4.7, NULL),
(5, 2, 7, 4.6, '2024-08-28 09:20:00'),
(6, 2, 8, 3.9, '2024-08-27 16:45:00'),
(7, 3, 1, 2.9, NULL),
(8, 3, 2, 4.9, '2024-08-26 12:30:00'),
(9, 3, 6, NULL, NULL);
""")
conn.commit()
conn.close()
runSql('Users', 'select * from Users')
runSql('Songs', 'select * from Songs')
runSql('Listens', 'select * from Listens')

user_id,name,email
1,Mickey,mickey@example.com
2,Mine,minnie@example.com
3,Daffy,daffy@example.com
4,Pluto,pluto@example.com


song_id,title,artist,genre
1,Evermore,Taylor Swift,Pop
2,Willow,Taylor Swift,Pop
3,Shape of You,Ed Sheeran,Rock
4,Photograph,Ed Sheeran,Rock
5,Shivers,Ed Sheeran,Rock
6,Yesterday,Beatles,Classic
7,Yellow Submarine,Beatles,Classic
8,Hey Jude,Beatles,Classic
9,Bad Blood,Taylor Swift,Rock
10,DJ Mix,DJ,None


listen_id,user_id,song_id,rating,listen_time
1,1,1,4.5,2024-08-30 14:35:00
2,1,2,4.2,None
3,1,6,3.9,2024-08-29 10:15:00
4,2,2,4.7,None
5,2,7,4.6,2024-08-28 09:20:00
6,2,8,3.9,2024-08-27 16:45:00
7,3,1,2.9,None
8,3,2,4.9,2024-08-26 12:30:00
9,3,6,NaN,None


# New Section

In [51]:
""" Goal: Learn basic forms of SELECT, FROM, WHERE, DISTINCT """
qry_classic_songs = """
---Find the title and artists of songs in the "Classic genre".
SELECT songs.title, Songs.artist FROM songs
Where Songs.genre = 'Classic';
"""
runSql('Classic songs', qry_classic_songs)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles
Hey Jude,Beatles


In [52]:
qry_classic2_songs = """
--- Find the title and artists of songs in the "Classic" genre.
SELECT songs.title, songs.artist
FROM songs
WHERE Songs.genre = 'Classic'
AND songs.title like 'Ye%';
"""
runSql('Classic songs starting from Ye', qry_classic2_songs)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles


In [53]:
qry_classic3_songs = """
--- Find the title and artists of songs in the "Classic" genre.
SELECT * FROM Songs

"""
runSql('Basic query', qry_classic3_songs)

song_id,title,artist,genre
1,Evermore,Taylor Swift,Pop
2,Willow,Taylor Swift,Pop
3,Shape of You,Ed Sheeran,Rock
4,Photograph,Ed Sheeran,Rock
5,Shivers,Ed Sheeran,Rock
6,Yesterday,Beatles,Classic
7,Yellow Submarine,Beatles,Classic
8,Hey Jude,Beatles,Classic
9,Bad Blood,Taylor Swift,Rock
10,DJ Mix,DJ,None


In [54]:
qry_classic4_songs = """
--- Find the title and artists of songs in the "Classic" genre.
SELECT Songs.title, Songs.artist, Songs.song_id, Songs.genre
FROM Songs
WHERE Songs.genre = 'Classic' OR
Songs.song_id < 3 ;

"""
runSql('Using Songs ID', qry_classic4_songs)

title,artist,song_id,genre
Evermore,Taylor Swift,1,Pop
Willow,Taylor Swift,2,Pop
Yesterday,Beatles,6,Classic
Yellow Submarine,Beatles,7,Classic
Hey Jude,Beatles,8,Classic


In [55]:
qry_genres = """
--- List of all genres in the Songs table
SELECT Songs.genre FROM Songs

"""
runSql('Genre Table Songs', qry_genres)

genre
Pop
Pop
Rock
Rock
Rock
Classic
Classic
Classic
Rock
None


In [56]:
qry_distinct = """
SELECT DISTINCT genre
FROM Songs;
"""
runSql('Distinct songs', qry_distinct)

genre
Pop
Rock
Classic
None


In [57]:
query_artistsong_count = """
--- Songs by the artist in differnet genre
SELECT genre, count(*) as num_songs
FROM songs
where artist = 'Taylor Swift'
GROUP BY genre;
"""
runSql('Count the songs genre', query_artistsong_count)

genre,num_songs
Pop,2
Rock,1


In [58]:
query_numsongs_artist = """
SELECT genre, count(*) as num_song
FROM Songs
GROUP BY genre;
"""
runSql('ALl the songs with count', query_numsongs_artist)

query_artist_count = """
SELECT genre, artist, count(*) as num_song
FROM Songs
GROUP BY genre, artist;
"""
runSql('ALl the songs with Artist', query_artist_count)


query_excluding_artist = """
SELECT genre, count(*) as num_songs
from songs
GROUP BY genre;
"""
runSql('Genre without artist', query_excluding_artist)

genre,num_song
None,1
Classic,3
Pop,2
Rock,4


genre,artist,num_song
None,DJ,1
Classic,Beatles,3
Pop,Taylor Swift,2
Rock,Ed Sheeran,3
Rock,Taylor Swift,1


genre,num_songs
None,1
Classic,3
Pop,2
Rock,4


In [59]:
query_all_the_tables = """
SELECT * FROM Users;
"""
runSql('Users tables', query_all_the_tables)
query_all_the_ListernsTable = """
SELECT * FROM Listens;
"""
runSql('Listeners tables', query_all_the_ListernsTable)
query_all_the_SongsTable = """
SELECT * FROM Songs;
"""
runSql('Listens Table', query_all_the_SongsTable)

user_id,name,email
1,Mickey,mickey@example.com
2,Mine,minnie@example.com
3,Daffy,daffy@example.com
4,Pluto,pluto@example.com


listen_id,user_id,song_id,rating,listen_time
1,1,1,4.5,2024-08-30 14:35:00
2,1,2,4.2,None
3,1,6,3.9,2024-08-29 10:15:00
4,2,2,4.7,None
5,2,7,4.6,2024-08-28 09:20:00
6,2,8,3.9,2024-08-27 16:45:00
7,3,1,2.9,None
8,3,2,4.9,2024-08-26 12:30:00
9,3,6,NaN,None


song_id,title,artist,genre
1,Evermore,Taylor Swift,Pop
2,Willow,Taylor Swift,Pop
3,Shape of You,Ed Sheeran,Rock
4,Photograph,Ed Sheeran,Rock
5,Shivers,Ed Sheeran,Rock
6,Yesterday,Beatles,Classic
7,Yellow Submarine,Beatles,Classic
8,Hey Jude,Beatles,Classic
9,Bad Blood,Taylor Swift,Rock
10,DJ Mix,DJ,None


In [60]:
query_left_join = """
SELECT * FROM songs
LEFT JOIN Listens ON Songs.song_id = Listens.Song_id
"""
runSql('Left Join', query_left_join)

song_id,title,artist,genre,listen_id,user_id,song_id,rating,listen_time
1,Evermore,Taylor Swift,Pop,1.0,1.0,1.0,4.5,2024-08-30 14:35:00
1,Evermore,Taylor Swift,Pop,7.0,3.0,1.0,2.9,None
2,Willow,Taylor Swift,Pop,2.0,1.0,2.0,4.2,None
2,Willow,Taylor Swift,Pop,4.0,2.0,2.0,4.7,None
2,Willow,Taylor Swift,Pop,8.0,3.0,2.0,4.9,2024-08-26 12:30:00
3,Shape of You,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None
4,Photograph,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None
5,Shivers,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None
6,Yesterday,Beatles,Classic,3.0,1.0,6.0,3.9,2024-08-29 10:15:00
6,Yesterday,Beatles,Classic,9.0,3.0,6.0,NaN,None


In [61]:
query_left_joins = """
SELECT * FROM Listens
LEFT JOIN Songs ON Listens.Song_id = Songs.song_id
"""
runSql('Left Join', query_left_joins)

listen_id,user_id,song_id,rating,listen_time,song_id,title,artist,genre
1,1,1,4.5,2024-08-30 14:35:00,1,Evermore,Taylor Swift,Pop
2,1,2,4.2,None,2,Willow,Taylor Swift,Pop
3,1,6,3.9,2024-08-29 10:15:00,6,Yesterday,Beatles,Classic
4,2,2,4.7,None,2,Willow,Taylor Swift,Pop
5,2,7,4.6,2024-08-28 09:20:00,7,Yellow Submarine,Beatles,Classic
6,2,8,3.9,2024-08-27 16:45:00,8,Hey Jude,Beatles,Classic
7,3,1,2.9,None,1,Evermore,Taylor Swift,Pop
8,3,2,4.9,2024-08-26 12:30:00,2,Willow,Taylor Swift,Pop
9,3,6,NaN,None,6,Yesterday,Beatles,Classic


In [62]:
query_join_left = """
SELECT * FROM Songs
LEFT JOIN Listens ON Songs.song_id = Listens.song_id
LEFT JOIN Users ON Listens.user_id = Users.user_id;
"""
runSql('Left join from the songs', query_join_left)

song_id,title,artist,genre,listen_id,user_id,song_id,rating,listen_time,user_id,name,email
1,Evermore,Taylor Swift,Pop,1.0,1.0,1.0,4.5,2024-08-30 14:35:00,1.0,Mickey,mickey@example.com
1,Evermore,Taylor Swift,Pop,7.0,3.0,1.0,2.9,None,3.0,Daffy,daffy@example.com
2,Willow,Taylor Swift,Pop,2.0,1.0,2.0,4.2,None,1.0,Mickey,mickey@example.com
2,Willow,Taylor Swift,Pop,4.0,2.0,2.0,4.7,None,2.0,Mine,minnie@example.com
2,Willow,Taylor Swift,Pop,8.0,3.0,2.0,4.9,2024-08-26 12:30:00,3.0,Daffy,daffy@example.com
3,Shape of You,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
4,Photograph,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
5,Shivers,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
6,Yesterday,Beatles,Classic,3.0,1.0,6.0,3.9,2024-08-29 10:15:00,1.0,Mickey,mickey@example.com
6,Yesterday,Beatles,Classic,9.0,3.0,6.0,NaN,None,3.0,Daffy,daffy@example.com


In [63]:
qry_rating_songs = """
SELECT * FROM Songs;

"""
runSql('Highly rated songs', qry_rating_songs)

qry_artist_table = """
SELECT * FROM Listens;
"""
runSql('Listeners table', qry_artist_table)

song_id,title,artist,genre
1,Evermore,Taylor Swift,Pop
2,Willow,Taylor Swift,Pop
3,Shape of You,Ed Sheeran,Rock
4,Photograph,Ed Sheeran,Rock
5,Shivers,Ed Sheeran,Rock
6,Yesterday,Beatles,Classic
7,Yellow Submarine,Beatles,Classic
8,Hey Jude,Beatles,Classic
9,Bad Blood,Taylor Swift,Rock
10,DJ Mix,DJ,None


listen_id,user_id,song_id,rating,listen_time
1,1,1,4.5,2024-08-30 14:35:00
2,1,2,4.2,None
3,1,6,3.9,2024-08-29 10:15:00
4,2,2,4.7,None
5,2,7,4.6,2024-08-28 09:20:00
6,2,8,3.9,2024-08-27 16:45:00
7,3,1,2.9,None
8,3,2,4.9,2024-08-26 12:30:00
9,3,6,NaN,None


In [64]:
qry_mickey_taylor_fan = """
SELECT Songs.song_id, Songs.title, Songs.artist, AVG(listens.rating)
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id

"""
runSql('All the tables', qry_mickey_taylor_fan)

song_id,title,artist,AVG(listens.rating)
1,Evermore,Taylor Swift,4.2


In [65]:
songs_rating_query = """
SELECT Songs.song_id, Songs.title, Songs.artist, Listens.rating FROM Songs
JOIN Listens ON Songs.song_id = Listens.listen_id
WHERE Listens.rating > 4.6

"""
runSql('Highly rated Songs', songs_rating_query)

song_id,title,artist,rating
4,Photograph,Ed Sheeran,4.7
8,Hey Jude,Beatles,4.9


In [66]:
Average_rating_query = """
SELECT Songs.song_id, Songs.title, Songs.artist, AVG(Listens.rating) FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
GROUP BY Songs.song_id, songs.title, songs.artist
"""
runSql('Average_rating_query ', Average_rating_query)

song_id,title,artist,AVG(Listens.rating)
1,Evermore,Taylor Swift,3.7
2,Willow,Taylor Swift,4.6
6,Yesterday,Beatles,3.9
7,Yellow Submarine,Beatles,4.6
8,Hey Jude,Beatles,3.9


In [67]:
Popular_songs = """
select songs.title, songs.artist, count(listens.song_id) as popular from songs
JOIN Listens ON songs.song_id = listens.song_id
GROUP BY songs.title, songs.artist
ORDER BY COUNT(listens.song_id) DESC
"""
runSql('Popular Songs ', Popular_songs)

title,artist,popular
Willow,Taylor Swift,3
Evermore,Taylor Swift,2
Yesterday,Beatles,2
Hey Jude,Beatles,1
Yellow Submarine,Beatles,1


In [68]:
taylor_swift = """
SELECT users.name, songs.title, songs.artist
from songs
JOIN listens ON songs.song_id = listens.song_id
JOIN users ON listens.user_id = users.user_id
where users.name = 'Mickey' AND songs.artist = 'Taylor Swift'
"""
runSql('taylor_swift', taylor_swift)

name,title,artist
Mickey,Evermore,Taylor Swift
Mickey,Willow,Taylor Swift


In [69]:
qry_songs = """
SELECT songs.title, songs.artist, listens.song_id from songs
JOIN listens ON songs.song_id = listens.song_id
GROUP BY songs.title, songs.artist
"""
runStepByStepSql(qry_songs, 4)

In [70]:
# query = """
# CREATE TABLE songs(id INTEGER, title TEXT);
# INSERT INTO songs VALUES (1, 'Song A');
# INSERT INTO songs VALUES (2, 'Song B');
# SELECT * FROM songs;
# """
# fromline = 1
# lines = query.strip().split('\n')
# for lineindex in range(1, len(lines)):
#   partial_query = '\n'.join(lines[:lineindex])
#   caption = 'Query till line:' + partial_query
#   # print(f'Lineidx={lineindex} -> partial_query:\n{partial_query}\n')
#   print(caption)

In [71]:
# for lineindex in range(1, len(lines)):
#   partial_query = '\n'.join(lines[:lineindex])
#   print(f'Lineidx={lineindex} -> partial_query:\n{partial_query}\n')

In [72]:
Average_rating_query = """
SELECT Songs.song_id, Songs.title, Songs.artist, AVG(Listens.rating) FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
GROUP BY Songs.song_id, songs.title, songs.artist
"""
runSql('Average_rating_query ', Average_rating_query)

song_id,title,artist,AVG(Listens.rating)
1,Evermore,Taylor Swift,3.7
2,Willow,Taylor Swift,4.6
6,Yesterday,Beatles,3.9
7,Yellow Submarine,Beatles,4.6
8,Hey Jude,Beatles,3.9
